In [3]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter, FFMpegWriter
from tqdm import tqdm
from scipy.signal import convolve2d

In [7]:
# Test perimeter term function

def perimeter_term(
                    source_point_value,
                    target_point_value,
                    target_point_index,
                    grid, 
                    new: bool):
    """
    Calculate the perimeter term.
    """
    kernel_8 = np.array([[1,1,1],
                     [1,0,1],
                     [1,1,1]], dtype=np.int32)


    def compute_perimeter_periodic(grid, value):
        """
        Compute perimeter of all pixels with a given value,
        using 8-connected neighbors and periodic boundaries.
        """
        mask = (grid == value).astype(np.int32)
        # Pad with wrap (periodic)
        mask_wrapped = np.pad(mask, pad_width=1, mode='wrap')
        # Convolve
        neighbor_count = convolve2d(mask_wrapped, kernel_8, mode='valid')
        
        # Each pixel contributes perimeter = #neighbors that are different
        perimeter_grid = 8 - neighbor_count
        return np.sum(perimeter_grid[mask == 1])
    

    grid_copy = grid.copy()
    if new: # Assume we are changing the target point to the source point
        grid_copy[target_point_index] = source_point_value
        
    P_source = compute_perimeter_periodic(grid_copy, source_point_value)
    P_target = compute_perimeter_periodic(grid_copy, target_point_value)
    
    
    return P_source, P_target

A = np.zeros((5,5))
A[0:2, 1] = 1
A[2:4, 2:4] = 3

print(A)

print(f"Expect 14 for P_1 and 20 for P_3:", perimeter_term(1, 3, (2,2), A, False))
print(f"Expect 20 for P_1 and 18 for P_3 after change of target:", perimeter_term(1, 3, (2,2), A, True))

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 3. 3. 0.]
 [0. 0. 3. 3. 0.]
 [0. 0. 0. 0. 0.]]
Expect 14 for P_1 and 20 for P_3: (14, 20)
Expect 20 for P_1 and 18 for P_3 after change of target: (20, 18)
